In [ ]:
import pandas as pd
from collections import defaultdict

# Read the CSV file into a DataFrame
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize empty groups and other variables
groups = defaultdict(list)
group_weights = defaultdict(int)
group_count = 9

# Calculate average weight for groups
avg_group_weight = df['Weight'].sum() / group_count

# Step 1: Seed groups with a member from each team
for team in df['Team'].unique():
    team_data = df[df['Team'] == team].sort_values(by='Weight', ascending=False)
    for idx, (group_id, player) in enumerate(team_data.iterrows()):
        group_number = idx % group_count
        groups[group_number].append(player['Name'])
        group_weights[group_number] += player['Weight']
        df.drop(index=group_id, inplace=True)  # Remove this player from df as they are now allocated

# Step 2: Fill remaining slots aiming for balanced groups
while not df.empty:
    # Sort groups by current weight to fill lighter groups first
    sorted_groups = sorted(group_weights.keys(), key=lambda x: group_weights[x])
    
    for group_number in sorted_groups:
        # If this group is already full, skip
        if len(groups[group_number]) >= 7:
            continue
            
        closest_player = None
        closest_diff = float('inf')
        
        # Find the player that brings the group closest to avg_group_weight
        for idx, player in df.iterrows():
            new_diff = abs((group_weights[group_number] + player['Weight']) - avg_group_weight)
            if new_diff < closest_diff:
                closest_diff = new_diff
                closest_player = player
                
        # Add the selected player to the group
        if closest_player is not None:
            groups[group_number].append(closest_player['Name'])
            group_weights[group_number] += closest_player['Weight']
            df.drop(index=closest_player.name, inplace=True)  # Remove this player from df as they are now allocated

# Display the groups
for group_number, group_members in groups.items():
    print(f"Group {group_number + 1}: {group_members} (Total Weight: {group_weights[group_number]})")


In [ ]:
import pandas as pd
from collections import defaultdict

# Read the CSV file into a DataFrame
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize empty groups and other variables
groups = defaultdict(list)
group_weights = defaultdict(int)
group_count = 9

# Calculate average weight for groups
avg_group_weight = df['Weight'].sum() / group_count

# Step 1: Seed groups with a member from each team
for team in df['Team'].unique():
    team_data = df[df['Team'] == team].sort_values(by='Weight', ascending=False)
    for idx, (group_id, player) in enumerate(team_data.iterrows()):
        group_number = idx % group_count
        groups[group_number].append(player['Name'])
        group_weights[group_number] += player['Weight']
        df.drop(index=group_id, inplace=True)  # Remove this player from df as they are now allocated

# Step 2: Fill remaining slots aiming for balanced groups
while not df.empty:
    # Sort groups by current weight to fill lighter groups first
    sorted_groups = sorted(group_weights.keys(), key=lambda x: group_weights[x])
    
    for group_number in sorted_groups:
        # If this group is already full, skip
        if len(groups[group_number]) >= 7:
            continue
            
        closest_player = None
        closest_diff = float('inf')
        
        # Find the player that brings the group closest to avg_group_weight
        for idx, player in df.iterrows():
            new_diff = abs((group_weights[group_number] + player['Weight']) - avg_group_weight)
            if new_diff < closest_diff:
                closest_diff = new_diff
                closest_player = player
                
        # Add the selected player to the group
        if closest_player is not None:
            groups[group_number].append(closest_player['Name'])
            group_weights[group_number] += closest_player['Weight']
            df.drop(index=closest_player.name, inplace=True)  # Remove this player from df as they are now allocated

# Create a DataFrame for the output
output_data = []

for group_number, group_members in groups.items():
    for member in group_members:
        output_data.append([f"Group {group_number + 1}", member, group_weights[group_number]])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name', 'Total Group Weight'])

# Write to CSV
output_df.to_csv('grouped_players.csv', index=False)


In [ ]:
import pandas as pd
from itertools import combinations

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size = 7
best_groups = None
best_variance = float('inf')

# Calculate the target weight for each group
target_weight = sum(player['Weight'] for player in players) // group_count

def find_groups(players, current_groups, current_weights):
    global best_variance, best_groups
    if not players:
        variance = max(current_weights) - min(current_weights)
        if variance < best_variance:
            best_variance = variance
            best_groups = [list(group) for group in current_groups]
        return
    
    player = players[0]
    for i, group in enumerate(current_groups):
        if len(group) < group_size and current_weights[i] + player['Weight'] <= target_weight:
            group.append(player['Name'])
            current_weights[i] += player['Weight']
            find_groups(players[1:], current_groups, current_weights)
            current_weights[i] -= player['Weight']
            group.pop()

# Initialize empty groups and weights
current_groups = [[] for _ in range(group_count)]
current_weights = [0 for _ in range(group_count)]

# Start finding groups
find_groups(players, current_groups, current_weights)

# Print and save to CSV
output_data = []
for i, group in enumerate(best_groups):
    print(f"Group {i + 1}: {group} (Total Weight: {sum(player['Weight'] for player in players if player['Name'] in group)})")
    for member in group:
        output_data.append([f"Group {i + 1}", member])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name'])
output_df.to_csv('grouped_players.csv', index=False)


In [ ]:
import pandas as pd
from collections import defaultdict

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size = 7
groups = [[] for _ in range(group_count)]
group_weights = [0 for _ in range(group_count)]

# Sort players by weight and team
players.sort(key=lambda x: (x['Team'], -x['Weight']))

# First pass to ensure at least 2 players of the same team in each group
for team in df['Team'].unique():
    team_players = [player for player in players if player['Team'] == team]
    for i in range(0, len(team_players), 2):
        pair = team_players[i:i + 2]
        target_group = min(range(group_count), key=lambda i: group_weights[i])
        for player in pair:
            groups[target_group].append(player['Name'])
            group_weights[target_group] += player['Weight']
            players.remove(player)

# Second pass to distribute remaining players
for player in players:
    target_group = min(range(group_count), key=lambda i: group_weights[i])
    groups[target_group].append(player['Name'])
    group_weights[target_group] += player['Weight']

# Create a DataFrame for the output
output_data = []
for i, group in enumerate(groups):
    for member in group:
        output_data.append([f"Group {i + 1}", member])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name'])

# Convert DataFrame to CSV format and print to console
csv_string = output_df.to_csv(index=False, header=True)
print("CSV Output:")
print(csv_string)


In [ ]:
import pandas as pd
from collections import defaultdict

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size = 7
groups = [[] for _ in range(group_count)]
group_weights = [0 for _ in range(group_count)]

# Sort players by weight and team
players.sort(key=lambda x: (x['Team'], -x['Weight']))

# First pass to ensure at least 2 players of the same team in each group
for team in df['Team'].unique():
    team_players = [player for player in players if player['Team'] == team]
    for i in range(0, len(team_players), 2):
        pair = team_players[i:i + 2]
        target_group = min(range(group_count), key=lambda i: group_weights[i])
        for player in pair:
            groups[target_group].append(player)
            group_weights[target_group] += player['Weight']
            players.remove(player)

# Second pass to distribute remaining players
for player in players:
    target_group = min(range(group_count), key=lambda i: group_weights[i])
    groups[target_group].append(player)
    group_weights[target_group] += player['Weight']

# Create a DataFrame for the output
output_data = []
for i, group in enumerate(groups):
    for player in group:
        output_data.append([f"Group {i + 1}", player['Name'], player['Team'], player['Weight']])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name', 'Team', 'Weight'])

# Convert DataFrame to CSV format and print to console
csv_string = output_df.to_csv(index=False, header=True)
print("CSV Output:")
print(csv_string)


In [ ]:
import pandas as pd
from collections import defaultdict

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size_upper = 7
group_size_lower = 6
groups = [[] for _ in range(group_count)]
group_weights = [0 for _ in range(group_count)]

# Sort players by weight and team
players.sort(key=lambda x: (x['Team'], -x['Weight']))

# First pass to ensure at least 2 players of the same team in each group
for team in df['Team'].unique():
    team_players = [player for player in players if player['Team'] == team]
    for i in range(0, len(team_players), 2):
        pair = team_players[i:i + 2]
        target_group = min(range(group_count), key=lambda i: group_weights[i])
        
        if len(groups[target_group]) + len(pair) <= group_size_upper:
            for player in pair:
                groups[target_group].append(player)
                group_weights[target_group] += player['Weight']
                players.remove(player)

# Second pass to distribute remaining players
for player in players:
    target_group = min(range(group_count), key=lambda i: group_weights[i])
    
    if len(groups[target_group]) < group_size_upper:
        groups[target_group].append(player)
        group_weights[target_group] += player['Weight']

# Ensure all groups have the same size (6 or 7)
while any(len(group) < group_size_lower for group in groups):
    smallest_group = min(range(group_count), key=lambda i: len(groups[i]))
    largest_group = max(range(group_count), key=lambda i: len(groups[i]))

    if len(groups[smallest_group]) < group_size_lower and len(groups[largest_group]) > group_size_lower:
        # Move a player from the largest to the smallest group while trying to minimize the impact on total group weight
        best_transfer = min(groups[largest_group], key=lambda x: x['Weight'])
        groups[smallest_group].append(best_transfer)
        groups[largest_group].remove(best_transfer)

        # Update group_weights
        group_weights[smallest_group] += best_transfer['Weight']
        group_weights[largest_group] -= best_transfer['Weight']

# Create a DataFrame for the output
output_data = []
for i, group in enumerate(groups):
    for player in group:
        output_data.append([f"Group {i + 1}", player['Name'], player['Team'], player['Weight']])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name', 'Team', 'Weight'])

# Convert DataFrame to CSV format and print to console
csv_string = output_df.to_csv(index=False, header=True)
print("CSV Output:")
print(csv_string)


In [ ]:
import pandas as pd
from collections import defaultdict

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size_upper = 7
group_size_lower = 6
groups = [[] for _ in range(group_count)]
group_weights = [0 for _ in range(group_count)]
placed_players = set()

# Sort players by weight and team
players.sort(key=lambda x: (x['Team'], -x['Weight']))

# First pass to ensure at least 2 players of the same team in each group
for team in df['Team'].unique():
    team_players = [player for player in players if player['Team'] == team]
    for i in range(0, len(team_players), 2):
        pair = team_players[i:i + 2]
        target_group = min(range(group_count), key=lambda i: group_weights[i])
        
        if len(groups[target_group]) + len(pair) <= group_size_upper:
            for player in pair:
                groups[target_group].append(player)
                group_weights[target_group] += player['Weight']
                placed_players.add(player['Name'])

# Second pass to distribute remaining players
remaining_players = [player for player in players if player['Name'] not in placed_players]
for player in remaining_players:
    target_group = min(range(group_count), key=lambda i: group_weights[i])
    
    if len(groups[target_group]) < group_size_upper:
        groups[target_group].append(player)
        group_weights[target_group] += player['Weight']

# Redistribution of players to ensure all groups have the same size (6 or 7)
while any(len(group) < group_size_lower for group in groups):
    smallest_group = min(range(group_count), key=lambda i: len(groups[i]))
    largest_group = max(range(group_count), key=lambda i: len(groups[i]))

    if len(groups[smallest_group]) < group_size_lower and len(groups[largest_group]) > group_size_lower:
        # Move a player from the largest to the smallest group while trying to minimize the impact on total group weight
        best_transfer = min(groups[largest_group], key=lambda x: x['Weight'])
        groups[smallest_group].append(best_transfer)
        groups[largest_group].remove(best_transfer)

        # Update group_weights
        group_weights[smallest_group] += best_transfer['Weight']
        group_weights[largest_group] -= best_transfer['Weight']

# Create a DataFrame for the output
output_data = []
for i, group in enumerate(groups):
    for player in group:
        output_data.append([f"Group {i + 1}", player['Name'], player['Team'], player['Weight']])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name', 'Team', 'Weight'])

# Convert DataFrame to CSV format and print to console
csv_string = output_df.to_csv(index=False, header=True)
print("CSV Output:")
print(csv_string)


In [ ]:
import pandas as pd
from collections import defaultdict

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size_upper = 7
group_size_lower = 6
groups = [[] for _ in range(group_count)]
group_weights = [0 for _ in range(group_count)]
placed_players = set()

# Sort players by weight and team
players.sort(key=lambda x: (x['Team'], -x['Weight']))

# First pass to ensure at least 2 players of the same team in each group
for team in df['Team'].unique():
    team_players = [player for player in players if player['Team'] == team]
    for i in range(0, len(team_players), 2):
        pair = team_players[i:i + 2]
        target_group = min(range(group_count), key=lambda i: group_weights[i])
        
        if len(groups[target_group]) + len(pair) <= group_size_upper:
            for player in pair:
                groups[target_group].append(player)
                group_weights[target_group] += player['Weight']
                placed_players.add(player['Name'])

# Second pass to distribute remaining players
remaining_players = [player for player in players if player['Name'] not in placed_players]
for player in remaining_players:
    target_group = min(range(group_count), key=lambda i: group_weights[i])
    
    if len(groups[target_group]) < group_size_upper:
        groups[target_group].append(player)
        group_weights[target_group] += player['Weight']

# Redistribution of players to ensure all groups have the same size (6 or 7)
while any(len(group) < group_size_lower for group in groups):
    smallest_group = min(range(group_count), key=lambda i: len(groups[i]))
    largest_group = max(range(group_count), key=lambda i: len(groups[i]))

    if len(groups[smallest_group]) < group_size_lower and len(groups[largest_group]) > group_size_lower:
        # Move a player from the largest to the smallest group while trying to minimize the impact on total group weight
        best_transfer = min(groups[largest_group], key=lambda x: x['Weight'])
        groups[smallest_group].append(best_transfer)
        groups[largest_group].remove(best_transfer)

        # Update group_weights
        group_weights[smallest_group] += best_transfer['Weight']
        group_weights[largest_group] -= best_transfer['Weight']

# Create a DataFrame for the output
output_data = []
for i, group in enumerate(groups):
    for player in group:
        output_data.append([f"Group {i + 1}", player['Name'], player['Team'], player['Weight']])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name', 'Team', 'Weight'])

# Convert DataFrame to CSV format and print to console
csv_string = output_df.to_csv(index=False, header=True)
print("CSV Output:")
print(csv_string)


In [ ]:
import pandas as pd

# Read CSV
df = pd.read_csv('olympiade_2023.csv', names=['Name', 'Team', 'Weight'])

# Initialize variables
players = df.to_dict('records')
group_count = 9
group_size_upper = 7
group_size_lower = 6
groups = [[] for _ in range(group_count)]
group_weights = [0 for _ in range(group_count)]

# Sort players by weight and team
players.sort(key=lambda x: (x['Team'], -x['Weight']))

# First pass to try to ensure at least 2 players of the same team in each group
for team in df['Team'].unique():
    team_players = [player for player in players if player['Team'] == team]
    for i in range(0, len(team_players)):
        player = team_players[i]
        target_group = min(range(group_count), key=lambda i: group_weights[i])

        if len(groups[target_group]) < group_size_upper:
            groups[target_group].append(player)
            group_weights[target_group] += player['Weight']
            players.remove(player)

# Second pass to distribute remaining players, if any
for player in players:
    target_group = min(range(group_count), key=lambda i: group_weights[i])
    if len(groups[target_group]) < group_size_upper:
        groups[target_group].append(player)
        group_weights[target_group] += player['Weight']

# Redistribution of players to ensure all groups have the same size (6 or 7)
while any(len(group) < group_size_lower for group in groups):
    smallest_group = min(range(group_count), key=lambda i: len(groups[i]))
    largest_group = max(range(group_count), key=lambda i: len(groups[i]))

    if len(groups[smallest_group]) < group_size_lower and len(groups[largest_group]) > group_size_lower:
        # Move a player from the largest to the smallest group while trying to minimize the impact on total group weight
        best_transfer = min(groups[largest_group], key=lambda x: x['Weight'])
        groups[smallest_group].append(best_transfer)
        groups[largest_group].remove(best_transfer)

        # Update group_weights
        group_weights[smallest_group] += best_transfer['Weight']
        group_weights[largest_group] -= best_transfer['Weight']

# Create a DataFrame for the output
output_data = []
for i, group in enumerate(groups):
    for player in group:
        output_data.append([f"Group {i + 1}", player['Name'], player['Team'], player['Weight']])

output_df = pd.DataFrame(output_data, columns=['Group', 'Name', 'Team', 'Weight'])

# Convert DataFrame to CSV format and print to console
csv_string = output_df.to_csv(index=False, header=True)
print("CSV Output:")
print(csv_string)


In [ ]:
import csv
from collections import defaultdict

def read_csv(input_str):
    csvreader = csv.reader(input_str.split('\n'))
    return [row for row in csvreader]

def group_players(data):
    team_dict = defaultdict(list)

    # Categorize players by their teams
    for row in data:
        name, team, weight = row
        weight = int(weight)
        team_dict[team].append((name, weight))

    # Sort players by weight within each team
    for team, players in team_dict.items():
        team_dict[team] = sorted(players, key=lambda x: x[1])

    groups = []
    group = []
    group_weight = 0

    for team, players in team_dict.items():
        for name, weight in players:
            if len(group) >= 7:
                groups.append(group)
                group = []
                group_weight = 0

            group.append((name, team, weight))
            group_weight += weight

    # Append the last group if it's not empty
    if group:
        groups.append(group)

    return groups

if __name__ == '__main__':
    # Replace the string input_str with your actual CSV input as a string
    input_str = """Alexia Bruyland,U14G,10
Mirthe Bruylandt,U14G,8
Flore Duyck,U14G,10
Noemie Fastenaekels,U14G,6
Linn Huylenbroeck,U14G,8
Lena Sack,U14G,6
Juliet Samyn,U14G,9
Marie Schockaert,U14G,8
Fien Van den Maegdenbergh,U14G,7
Kato Van den Maegdenbergh,U14G,7
Stien Veldeman,U14G,6
Quinten Asselman,U12B,4
Berre Baeke,U12B,5
Niklas Braeckevelt,U12B,7
Marcel De Ketelaere,U12B,8
Finn Van Gheluwe,U12B,10
Michiel Verhelst,U12B,9
Gus Buyse,U12B,8
Arthur Heirweg,U12B,7
Lou Samyn,U12B,8
Finn Spitters,U12B,5
Eppo Van Bael,U12B,5
Manu Van Hecke,U12B,6
Hannes Vandebergh,U12B,5
Kasper Verbessselt,U12B,4
Jef Westerlinck,U12B,7
Lili Duyck,U11G,6
Feline Govaert,U11G,6
Lily Hofman,U11G,6
Laurien De Keyzer,U11G,5
Rose Hofman,U11G,6
Feline Huyck,U11G,6
Irene Sack,U11G,7
Nell Van Cauteren,U11G,7
Nore Van den Maegdenbergh,U11G,8
Louise Van Kemenade,U11G,5
Julie Van Praet,U11G,5
Marie Verbeiren,U11G,5
Elle-Nore Verdyck,U11G,3
Gus Baeke,U10B,3
Lou Vermeir,U10B,6
Achille De Ketelaere,U10B,7
Ferren D'hondt,U10B,4
Mathis Fransen,U10B,5
Thor Van Driessche,U10B,7
Jonathan Van Gaeveren,U10B,3
Jules Westerlinck,U10B,8
Gil Saerens Nogueira,U10B,7
Alix De Feyter,U9,4
Juul De Bock,U9,3
Flor Germonprez,U9,4
Lauren Hebbelinck,U9,3
Felix Van Boeckel,U9,4
Jef Van De Wiele,U9,4
Noud Van Gheluwe,U9,5
Lio Janssens,U9,4
Max Van Praet,U9,4
Quinn Saerens Nogueira,U9,5
Oda Van Bael,U9,4
Willem Verhelst,U9,3
August Wetzels,U9,3
Mats Van Dienderen,U9,3
Anna Vannyvel,U9,3"""

    input_data = read_csv(input_str)

    # Group players
    groups = group_players(input_data)

    # Generate output
    print("Group,Name,Team,Weight")
    group_id = 1

    for group in groups:
        for name, team, weight in group:
            print(f"{group_id},{name},{team},{weight}")
        group_id += 1


In [ ]:
from collections import defaultdict
import csv

def read_csv(input_str):
    csvreader = csv.reader(input_str.split('\n'))
    return [row for row in csvreader]

def create_groups(players):
    players.sort(key=lambda x: (x[1], -x[2]))
    
    groups = []
    current_group = []
    current_weight = 0

    for player in players:
        name, team, weight = player
        if len(current_group) < 7:
            current_group.append(player)
            current_weight += weight
        else:
            # Check if adding this player would make this group's weight closer to the next group's weight
            next_group_weight = sum(p[2] for p in current_group[1:])
            if abs(current_weight - next_group_weight) > abs(current_weight + weight - next_group_weight):
                current_group.append(player)
                current_weight += weight
            else:
                groups.append(current_group)
                current_group = [player]
                current_weight = weight

    if current_group:
        groups.append(current_group)
        
    return groups

if __name__ == '__main__':
    input_str = """Alexia Bruyland,U14G,10
Mirthe Bruylandt,U14G,8
Flore Duyck,U14G,10
Noemie Fastenaekels,U14G,6
Linn Huylenbroeck,U14G,8
Lena Sack,U14G,6
Juliet Samyn,U14G,9
Marie Schockaert,U14G,8
Fien Van den Maegdenbergh,U14G,7
Kato Van den Maegdenbergh,U14G,7
Stien Veldeman,U14G,6
Quinten Asselman,U12B,4
Berre Baeke,U12B,5
Niklas Braeckevelt,U12B,7
Marcel De Ketelaere,U12B,8
Finn Van Gheluwe,U12B,10
Michiel Verhelst,U12B,9
Gus Buyse,U12B,8
Arthur Heirweg,U12B,7
Lou Samyn,U12B,8
Finn Spitters,U12B,5
Eppo Van Bael,U12B,5
Manu Van Hecke,U12B,6
Hannes Vandebergh,U12B,5
Kasper Verbessselt,U12B,4
Jef Westerlinck,U12B,7
Lili Duyck,U11G,6
Feline Govaert,U11G,6
Lily Hofman,U11G,6
Laurien De Keyzer,U11G,5
Rose Hofman,U11G,6
Feline Huyck,U11G,6
Irene Sack,U11G,7
Nell Van Cauteren,U11G,7
Nore Van den Maegdenbergh,U11G,8
Louise Van Kemenade,U11G,5
Julie Van Praet,U11G,5
Marie Verbeiren,U11G,5
Elle-Nore Verdyck,U11G,3
Gus Baeke,U10B,3
Lou Vermeir,U10B,6
Achille De Ketelaere,U10B,7
Ferren D'hondt,U10B,4
Mathis Fransen,U10B,5
Thor Van Driessche,U10B,7
Jonathan Van Gaeveren,U10B,3
Jules Westerlinck,U10B,8
Gil Saerens Nogueira,U10B,7
Alix De Feyter,U9,4
Juul De Bock,U9,3
Flor Germonprez,U9,4
Lauren Hebbelinck,U9,3
Felix Van Boeckel,U9,4
Jef Van De Wiele,U9,4
Noud Van Gheluwe,U9,5
Lio Janssens,U9,4
Max Van Praet,U9,4
Quinn Saerens Nogueira,U9,5
Oda Van Bael,U9,4
Willem Verhelst,U9,3
August Wetzels,U9,3
Mats Van Dienderen,U9,3
Anna Vannyvel,U9,3"""
    input_data = read_csv(input_str)

    # Remove empty lines and convert weights to integers
    input_data = [(name, team, int(weight)) for name, team, weight in input_data if name]

    groups = create_groups(input_data)

    print("Group,Name,Team,Weight")
    
    for i, group in enumerate(groups, start=1):
        for name, team, weight in group:
            print(f"{i},{name},{team},{weight}")
